## Проект: вариант 3
## Представьте, что вы работаете в крупном дейтинговом приложении.

## Помимо базовых функций, в приложении также имеется премиум-подписка, которая дает доступ к ряду важных дополнительных возможностей. Был проведен A/B тест, в рамках которого для новых пользователей из нескольких стран была изменена стоимость премиум-подписки* при покупке через две новые платежные системы. При этом стоимость пробного периода оставалась прежней.

## Проверьте:

## Был ли эксперимент успешен в целом.
#### *Деньги за подписку списываются ежемесячно до тех пор, пока пользователь её не отменит.

Данные
Всего есть три группы: тестовая (test), контрольная 1 (control_1) и контрольная 2 (control_2). Для каждой из них:
users_*.csv – информация о пользователях:
uid – идентификатор пользователя
age – возрастattraction_coeff – коэффициент привлекательности (от 0 до 1000, \frac{лайки}{просмотры}*1000 
просмотры/лайки  ∗1000)
coins – число монеток (внутренняя валюта)
country – страна  
visit_days – в какие дни после регистрации пользователь посещал приложение (напр. в 1, затем в 7)
gender – пол
age_filter_start  – фильтр поиска, мин. значение 
age_filter_end  – фильтр поиска, макс. значение 
views_count – число полученных оценок 
was_premium – был ли когда-либо премиум (либо пробный период премиум-статуса, либо купленный за деньги)
is_premium –  является ли премиум
total_revenue – нормированная выручка 

transactions_*.csv – информация о платежах пользователей:
uid – идентификатор пользователя
country – страна
joined_at – дата и время регистрации
paid_at – дата и время покупки
revenue – нормированная выручка
payment_id – идентификатор платежа
from_page – откуда пользователь перешел на страницу оплаты
product_type – тип продукта (trial_premium – пробная премиум-подписка, premium_no_trial – премиум-подписка без пробной, coins – подписка за внутреннюю валюту, other_type – другое) 

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import levene
from matplotlib import pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

Загружаем  необходимые файлы

In [2]:
users_test = pd.read_csv('Проект_3_users_test.csv', sep=';')
users_test

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892309896,27,685,NaN,United States of America,"1,2,3,4,5,6,7,8,9,10,11,12,16,17",1,24,30,89,NaN,NaN,0
1,892044516,27,0,NaN,Germany,NaN,1,24,30,0,NaN,NaN,0
2,892185708,45,44,NaN,Israel,"1,3,4,6,7,8,10,11,12,13,14,15,16,17,18,19,20,2...",1,42,48,68,NaN,NaN,0
3,892130292,32,0,NaN,United States of America,"1,2,3,4,5,6,7,8,9,10,11,12",1,29,35,0,NaN,NaN,0
4,891406035,27,1000,NaN,France,NaN,1,24,30,1,1.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4303,891078639,17,895,NaN,United States of America,"1,2,3,4,5",1,16,20,96,NaN,NaN,0
4304,891188733,27,866,NaN,United States of America,"1,2,3",1,18,41,30,NaN,NaN,0
4305,891293718,48,0,NaN,Argentina,NaN,1,47,51,10,NaN,NaN,0
4306,891192357,32,0,NaN,Australia,NaN,1,16,49,0,NaN,NaN,0


Подгружаем все датафреймы, проверяем на наличие пропусков и дупликатов

In [3]:
users_test.info()
users_test.duplicated(keep=False).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308 entries, 0 to 4307
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uid               4308 non-null   int64  
 1   age               4308 non-null   int64  
 2   attraction_coeff  4308 non-null   int64  
 3   coins             49 non-null     float64
 4   country           4308 non-null   object 
 5   visit_days        2978 non-null   object 
 6   gender            4308 non-null   int64  
 7   age_filter_start  4308 non-null   int64  
 8   age_filter_end    4308 non-null   int64  
 9   views_count       4308 non-null   int64  
 10  was_premium       408 non-null    float64
 11  is_premium        157 non-null    float64
 12  total_revenue     4308 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 437.7+ KB


np.int64(0)

In [4]:
users_control_1  = pd.read_csv('Проект_3_users_control_1.csv', sep=';')
users_control_1 

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892319115,25,435,NaN,United States of America,"1,2,3,4,5,6",0,22,32,982,NaN,NaN,0
1,891248523,29,500,NaN,United States of America,"1,2",1,26,32,12,NaN,NaN,0
2,891670932,33,800,NaN,France,"1,2,3,5,6,8,9,11,13,16",1,30,36,5,NaN,NaN,0
3,891060786,26,0,NaN,Argentina,"1,2,3,4,5",1,23,29,0,NaN,NaN,0
4,892006554,35,294,NaN,United States of America,"1,2,3,5,6,7,8,9,10,12,15,16,17,19",1,30,40,17,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,892022304,26,227,NaN,United States of America,NaN,0,23,29,427,NaN,NaN,0
4336,891905022,38,200,NaN,Argentina,"1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,21",1,20,50,5,NaN,NaN,0
4337,891304722,49,411,NaN,United States of America,1,1,46,52,17,NaN,NaN,0
4338,891302328,27,0,NaN,United States of America,"1,3,5",1,24,30,0,NaN,NaN,0


In [5]:
users_control_1.info()
users_control_1.duplicated(keep=False).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uid               4340 non-null   int64  
 1   age               4340 non-null   int64  
 2   attraction_coeff  4340 non-null   int64  
 3   coins             60 non-null     float64
 4   country           4340 non-null   object 
 5   visit_days        3016 non-null   object 
 6   gender            4340 non-null   int64  
 7   age_filter_start  4340 non-null   int64  
 8   age_filter_end    4340 non-null   int64  
 9   views_count       4340 non-null   int64  
 10  was_premium       436 non-null    float64
 11  is_premium        192 non-null    float64
 12  total_revenue     4340 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 440.9+ KB


np.int64(0)

In [6]:
users_control_2  = pd.read_csv('Проект_3_users_control_2.csv', sep=';')
users_control_2

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892035504,37,137,NaN,Israel,"1,2,3,4,5,6,7,8,9,11,13",1,30,40,51,NaN,NaN,0
1,891782112,57,0,NaN,Italy,1,1,54,60,0,NaN,NaN,0
2,891110337,30,769,NaN,France,"1,2,3,4,5",1,27,33,13,NaN,NaN,0
3,891796320,48,750,NaN,France,"1,4,6,8,10,11,12,14,15,16,18",1,45,51,12,NaN,NaN,0
4,891880212,54,638,NaN,United States of America,1,1,35,53,94,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4259,891937401,63,242,NaN,Italy,"1,2",1,50,66,33,NaN,NaN,0
4260,891692844,35,500,NaN,France,"1,2,3,4,5,6,8,9,10,11,12,14,16",1,24,45,2,NaN,NaN,0
4261,891096429,41,333,NaN,United States of America,NaN,1,38,44,6,NaN,NaN,0
4262,891697584,48,210,NaN,United Kingdom (Great Britain),"2,3,5,7,9,10,12,14,15",1,45,51,19,NaN,NaN,0


In [7]:
users_control_2.info()
users_control_2.duplicated(keep=False).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4264 entries, 0 to 4263
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uid               4264 non-null   int64  
 1   age               4264 non-null   int64  
 2   attraction_coeff  4264 non-null   int64  
 3   coins             61 non-null     float64
 4   country           4264 non-null   object 
 5   visit_days        2900 non-null   object 
 6   gender            4264 non-null   int64  
 7   age_filter_start  4264 non-null   int64  
 8   age_filter_end    4264 non-null   int64  
 9   views_count       4264 non-null   int64  
 10  was_premium       411 non-null    float64
 11  is_premium        191 non-null    float64
 12  total_revenue     4264 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 433.2+ KB


np.int64(0)

In [8]:
transactions_test   = pd.read_csv('Проект_3_transactions_test.csv', sep=';')
transactions_test

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891345942,Italy,2017-05-11 13:00:00,2017-11-13 15:04:00,12909,147,trial_vip_popup,trial_premium
1,892054251,United States of America,2017-10-22 00:33:00,2017-10-30 01:37:00,13923,147,trial_vip_popup,trial_premium
2,892236423,United States of America,2017-10-18 01:09:00,2017-10-23 00:15:00,3783,67,menu,other_type
3,892236423,United States of America,2017-10-18 01:09:00,2017-10-27 22:38:00,3783,67,menu,other_type
4,892168170,United States of America,2017-10-19 17:10:00,2017-10-27 19:10:00,9087,147,trial_vip_popup,trial_premium
...,...,...,...,...,...,...,...,...
268,891419625,Israel,2017-04-11 05:58:00,2017-11-11 08:33:00,14352,147,trial_vip_popup,trial_premium
269,891447105,United Kingdom (Great Britain),2017-03-11 17:05:00,2017-11-11 04:23:00,7319,147,fans,premium_no_trial
270,891497073,United States of America,2017-02-11 15:33:00,2017-05-11 04:06:00,2730,67,empty_likes,other_type
271,891406224,Israel,2017-04-11 12:56:00,2017-04-11 13:35:00,2743,67,empty_likes,other_type


In [9]:
transactions_test.info()
transactions_test.duplicated(keep=False).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uid           273 non-null    int64 
 1   country       273 non-null    object
 2   joined_at     273 non-null    object
 3   paid_at       273 non-null    object
 4   revenue       273 non-null    int64 
 5   payment_id    273 non-null    int64 
 6   from_page     273 non-null    object
 7   product_type  273 non-null    object
dtypes: int64(3), object(5)
memory usage: 17.2+ KB


np.int64(14)

In [10]:
transactions_control_1    = pd.read_csv('Проект_3_transactions_control_1.csv', sep=';')
transactions_control_1 

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891319275.0,France,2017-05-11 19:57:00,2017-12-11 21:57:00,12727.0,147.0,trial_vip_popup,trial_premium
1,892421826.0,Israel,2017-10-14 12:46:00,2017-10-23 12:54:00,14586.0,147.0,trial_vip_popup,trial_premium
2,891822480.0,Italy,2017-10-26 22:27:00,2017-12-11 20:59:00,1911.0,19.0,none,coins
3,891367521.0,Italy,2017-05-11 02:37:00,2017-06-11 19:15:00,1456.0,127.0,sympathy,other_type
4,892207959.0,France,2017-10-18 18:30:00,2017-12-11 12:03:00,1261.0,19.0,empty_likes,coins
...,...,...,...,...,...,...,...,...
1002,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1003,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1004,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1005,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [11]:
transactions_control_1.info()
transactions_control_1.duplicated(keep=False).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uid           377 non-null    float64
 1   country       377 non-null    object 
 2   joined_at     1007 non-null   object 
 3   paid_at       1007 non-null   object 
 4   revenue       377 non-null    float64
 5   payment_id    377 non-null    float64
 6   from_page     377 non-null    object 
 7   product_type  377 non-null    object 
dtypes: float64(3), object(5)
memory usage: 63.1+ KB


np.int64(668)

In [12]:
transactions_control_2    = pd.read_csv('Проект_3_transactions_control_2.csv', sep=';')
transactions_control_2 

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891266616,Argentina,2017-06-11 15:25:00,2017-10-11 17:35:00,6305,19,autorefill,coins
1,892186737,Chile,2017-10-19 06:03:00,2017-04-11 08:53:00,4732,147,promo_09,premium_no_trial
2,891959004,United States of America,2017-10-24 00:12:00,2017-10-31 02:12:00,10153,147,trial_vip_popup,trial_premium
3,892115478,Spain,2017-10-20 20:39:00,2017-10-26 06:26:00,2366,67,empty_likes,other_type
4,891592941,Switzerland,2017-10-31 12:40:00,2017-03-11 16:25:00,6292,147,promo_09,premium_no_trial
...,...,...,...,...,...,...,...,...
323,891985866,Belgium,2017-10-23 13:19:00,2017-10-30 15:35:00,16640,147,trial_vip_popup,trial_premium
324,891491145,United States of America,2017-02-11 18:13:00,2017-09-11 20:32:00,12727,147,send_message,trial_premium
325,892053750,United States of America,2017-10-22 00:46:00,2017-10-28 08:17:00,1261,19,autorefill,coins
326,891668253,Chile,2017-10-29 23:08:00,2017-11-15 00:57:00,9997,147,trial_vip_popup,trial_premium


In [13]:
transactions_control_2.info()
transactions_control_2.duplicated(keep=False).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uid           328 non-null    int64 
 1   country       328 non-null    object
 2   joined_at     328 non-null    object
 3   paid_at       328 non-null    object
 4   revenue       328 non-null    int64 
 5   payment_id    328 non-null    int64 
 6   from_page     328 non-null    object
 7   product_type  328 non-null    object
dtypes: int64(3), object(5)
memory usage: 20.6+ KB


np.int64(10)

Проверяем нет ли пересечений между группами

In [14]:
users_test_set = set(users_test['uid'].unique()) 
users_control_1_set = set(users_control_1['uid'].unique())
users_test_set.intersection(users_control_1_set)

set()

In [15]:
users_control_2_set = set(users_control_2['uid'].unique())
users_control_1_set = set(users_control_1['uid'].unique())
users_test_set.intersection(users_control_2_set)

set()

Сравниваме группы по размеру

In [16]:
users_test.uid.nunique()

4308

In [17]:
users_control_1.uid.nunique()

4340

In [18]:
users_control_2.uid.nunique()

4264

Меняем пропуски на ноль

In [19]:
users_test = users_test.fillna(0)
users_control_1 = users_control_1.fillna(0)
users_control_2 = users_control_2.fillna(0)
transactions_control_1 =transactions_control_1.fillna(0)

Удаляем дупликаты

In [20]:
transactions_test.drop_duplicates(inplace = True) 
transactions_control_1.drop_duplicates(inplace = True) 
transactions_control_2.drop_duplicates(inplace = True) 

In [21]:
fdf = transactions_control_1[transactions_control_1.paid_at	 == 'NaT']
fdf

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
377,0.0,0,NaT,NaT,0.0,0.0,0,0


In [22]:
transactions_control_1_filt = transactions_control_1.drop(index=377)


Филтруем данные по дате оплаты которые не входят в период эксперимента

In [23]:
transactions_control_1_filt = transactions_control_1_filt[transactions_control_1_filt.paid_at >= '2017-01-11' ]

In [24]:
transactions_test.paid_at.min()


'2017-01-11 04:37:00'

In [25]:
transactions_test.paid_at.max()

'2017-12-11 22:06:00'

In [26]:
transactions_control_1_filt.paid_at.max()

'2017-12-11 21:57:00'

In [27]:
transactions_control_1_filt.paid_at.min()

'2017-01-11 02:56:00'

In [28]:
transactions_control_2.paid_at.min()

'2017-01-11 05:34:00'

In [29]:
transactions_control_2.paid_at.max()

'2017-12-11 23:28:00'

Создаем список стран, которые пересекаются по всем 6ти датафреймам

In [30]:
country_ls = list(transactions_test.query('country in @transactions_control_1.country.unique() and country in @transactions_control_2.country.unique() and country in @users_control_1.country.unique() and country in @users_control_2.country.unique() and country in @users_test.country.unique()').country.unique())

In [31]:
country_ls

['Italy',
 'United States of America',
 'Canada',
 'Spain',
 'United Arab Emirates',
 'Argentina',
 'France',
 'United Kingdom (Great Britain)',
 'Israel',
 'Switzerland',
 'Germany',
 'Belgium',
 'Chile',
 'Australia',
 'Turkey']

Филтруем все ДФ по общему списку стан присутствующих во всех датафреймах

In [32]:
users_control_1= users_control_1.query('country in @country_ls')


In [34]:
users_control_2= users_control_2.query('country in @country_ls')

In [35]:
users_test = users_test.query('country in @country_ls')


In [37]:
transactions_control_1_filt = transactions_control_1_filt.query('country in @country_ls')

In [38]:
transactions_control_2_filt = transactions_control_2.query('country in @country_ls')

In [39]:
transactions_test  = transactions_test.query('country in @country_ls')

Определяем по каким ПС проводилось тестирование

In [40]:
transactions_control_1_filt[transactions_control_1_filt.product_type == 'premium_no_trial'].payment_id.value_counts()

payment_id
147.0    85
68.0     16
0.0       2
Name: count, dtype: int64

In [41]:
transactions_control_2_filt[transactions_control_2_filt.product_type == 'premium_no_trial'].payment_id.value_counts()

payment_id
147    88
68     11
0       1
Name: count, dtype: int64

In [42]:
transactions_test[transactions_test.product_type == 'premium_no_trial'].payment_id.value_counts()

payment_id
147    61
68     12
0       1
Name: count, dtype: int64

#### Вывод: эксперимент проводился на ПС 147 и 68

Филтруем данные по тестовым платежным системам

In [44]:
transactions_control_1_filt['payment_id'] = transactions_control_1_filt['payment_id'].astype(str)

In [45]:
transactions_control_1_filt= transactions_control_1_filt.query('payment_id == "147.0" or payment_id == "68.0"')

In [46]:
transactions_control_1_filt

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891319275.0,France,2017-05-11 19:57:00,2017-12-11 21:57:00,12727.0,147.0,trial_vip_popup,trial_premium
1,892421826.0,Israel,2017-10-14 12:46:00,2017-10-23 12:54:00,14586.0,147.0,trial_vip_popup,trial_premium
8,892380396.0,Australia,2017-10-15 09:02:00,2017-10-23 13:27:00,13403.0,147.0,trial_vip_popup,trial_premium
10,892210755.0,Chile,2017-10-18 17:18:00,2017-10-25 19:29:00,9971.0,147.0,trial_vip_popup,trial_premium
15,891131061.0,Chile,2017-09-11 07:04:00,2017-09-11 23:38:00,6292.0,147.0,left_sidebar_menu,premium_no_trial
...,...,...,...,...,...,...,...,...
371,892074894.0,Canada,2017-10-21 17:12:00,2017-10-25 17:20:00,6292.0,147.0,promo_09,premium_no_trial
372,891556971.0,France,2017-01-11 05:07:00,2017-01-11 05:16:00,2886.0,68.0,none,premium_no_trial
373,892052106.0,Spain,2017-10-22 01:38:00,2017-10-29 03:44:00,12883.0,147.0,trial_vip_popup,trial_premium
374,892145004.0,Chile,2017-10-20 04:25:00,2017-01-11 02:56:00,9919.0,147.0,trial_vip_popup,trial_premium


In [48]:
transactions_control_2_filt

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891266616,Argentina,2017-06-11 15:25:00,2017-10-11 17:35:00,6305,19,autorefill,coins
1,892186737,Chile,2017-10-19 06:03:00,2017-04-11 08:53:00,4732,147,promo_09,premium_no_trial
2,891959004,United States of America,2017-10-24 00:12:00,2017-10-31 02:12:00,10153,147,trial_vip_popup,trial_premium
3,892115478,Spain,2017-10-20 20:39:00,2017-10-26 06:26:00,2366,67,empty_likes,other_type
4,891592941,Switzerland,2017-10-31 12:40:00,2017-03-11 16:25:00,6292,147,promo_09,premium_no_trial
...,...,...,...,...,...,...,...,...
323,891985866,Belgium,2017-10-23 13:19:00,2017-10-30 15:35:00,16640,147,trial_vip_popup,trial_premium
324,891491145,United States of America,2017-02-11 18:13:00,2017-09-11 20:32:00,12727,147,send_message,trial_premium
325,892053750,United States of America,2017-10-22 00:46:00,2017-10-28 08:17:00,1261,19,autorefill,coins
326,891668253,Chile,2017-10-29 23:08:00,2017-11-15 00:57:00,9997,147,trial_vip_popup,trial_premium


In [49]:
transactions_control_2_filt['payment_id'] = transactions_control_2_filt['payment_id'].astype(str)



C:\Users\Анна\AppData\Local\Temp\ipykernel_1700\1375474436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_control_2_filt['payment_id'] = transactions_control_2_filt['payment_id'].astype(str)


In [50]:
transactions_control_2_filt= transactions_control_2_filt.query('payment_id == "147" or payment_id == "68"')

In [51]:
transactions_control_2_filt

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
1,892186737,Chile,2017-10-19 06:03:00,2017-04-11 08:53:00,4732,147,promo_09,premium_no_trial
2,891959004,United States of America,2017-10-24 00:12:00,2017-10-31 02:12:00,10153,147,trial_vip_popup,trial_premium
4,891592941,Switzerland,2017-10-31 12:40:00,2017-03-11 16:25:00,6292,147,promo_09,premium_no_trial
5,892245918,France,2017-10-17 21:05:00,2017-10-17 22:35:00,6292,147,autoreply_message,premium_no_trial
9,892080081,Switzerland,2017-10-21 15:15:00,2017-10-22 16:37:00,4758,147,autoreply_message,premium_no_trial
...,...,...,...,...,...,...,...,...
322,891435993,Canada,2017-03-11 21:08:00,2017-03-11 21:26:00,3549,68,none,premium_no_trial
323,891985866,Belgium,2017-10-23 13:19:00,2017-10-30 15:35:00,16640,147,trial_vip_popup,trial_premium
324,891491145,United States of America,2017-02-11 18:13:00,2017-09-11 20:32:00,12727,147,send_message,trial_premium
326,891668253,Chile,2017-10-29 23:08:00,2017-11-15 00:57:00,9997,147,trial_vip_popup,trial_premium


In [52]:
transactions_test['payment_id'] = transactions_test['payment_id'].astype(str)

C:\Users\Анна\AppData\Local\Temp\ipykernel_1700\715483780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_test['payment_id'] = transactions_test['payment_id'].astype(str)


In [53]:
transactions_test= transactions_test.query('payment_id == "147" or payment_id == "68"')

Для правильного расчета конверсии фильтруем ДФ с юзерами, что бы в них были только юзеры пользующиеся тестовыми платежными системами

In [55]:
users_test      = users_test.query('uid in @transactions_test.uid.unique()')
users_control_1 = users_control_1.query('uid in @transactions_control_1.uid.unique()')
users_control_2 = users_control_2.query('uid in @transactions_control_2.uid.unique()')

#### Сравниваем  распределение ключевых признаков (возраст, пол) между группами, что бы понимать, что пользователи у нас для проведения АБ-теста были отобраны корректно

In [56]:
group_test = users_test['gender']
group_C1 = users_control_1['gender']
group_C2= users_control_1['gender']

In [57]:
statistic, p_value = stats.kruskal(group_test, group_C1, group_C2)
print(f"Статистика теста: {statistic}")
print(f"P-значение: {p_value}")

Статистика теста: 0.14861857256543556
P-значение: 0.9283845128380955


#### Вывод: так как p_value > 0.05, не отклоняем нулевую гипотезу- нет статистически значимых различий в распределении пола между группами

In [58]:
group_test_age = users_test['age']
group_C1_age = users_control_1['age']
group_C2_age= users_control_1['age']

In [59]:
statistic, p_value = stats.kruskal(group_test_age, group_C1_age, group_C2_age)
print(f"Статистика теста: {statistic}")
print(f"P-значение: {p_value}")

Статистика теста: 2.6420173048667603
P-значение: 0.26686599113682496


#### Вывод: так как p_value > 0.05, не отклоняем нулевую гипотезу, следовательно нет статистически значимых различий в распределении возроста между группами

####  Проводим тест хи-квадрат для категориальных переменных (например, страна) для проверки отсутствия статистически значимых различий между группами.

Делаем сводную таблицу сопряженности, для теста хи-квадрат

In [60]:
u_control_1 = users_control_1.groupby('country', as_index = False).agg({'gender':'count'})
u_control_1 = u_control_1.assign(group='control_1')
u_control_1= u_control_1.pivot_table(index='group', columns='country', values='gender', fill_value=0)
u_control_1

country,Argentina,Australia,Belgium,Canada,Chile,France,Germany,Israel,Italy,Spain,Switzerland,Turkey,United Arab Emirates,United Kingdom (Great Britain),United States of America
group,,,,,,,,,,,,,,,
control_1,12.0,3.0,4.0,9.0,10.0,22.0,5.0,14.0,17.0,9.0,3.0,3.0,7.0,10.0,60.0


In [61]:
u_control_2 = users_control_2.groupby('country', as_index = False).agg({'gender':'count'})
u_control_2 = u_control_2.assign(group='control_2')
u_control_2= u_control_2.pivot_table(index='group', columns='country', values='gender', fill_value=0)
u_control_2

country,Argentina,Australia,Belgium,Canada,Chile,France,Germany,Israel,Italy,Spain,Switzerland,Turkey,United Arab Emirates,United Kingdom (Great Britain),United States of America
group,,,,,,,,,,,,,,,
control_2,10.0,2.0,10.0,4.0,11.0,19.0,5.0,6.0,16.0,12.0,3.0,1.0,5.0,10.0,64.0


In [62]:
u_test = users_test.groupby('country', as_index = False).agg({'gender':'count'})
u_test = u_test.assign(group='test')
u_test= u_test.pivot_table(index='group', columns='country', values='gender', fill_value=0)
u_test


country,Argentina,Australia,Belgium,Canada,Chile,France,Germany,Israel,Italy,Spain,Switzerland,Turkey,United Arab Emirates,United Kingdom (Great Britain),United States of America
group,,,,,,,,,,,,,,,
test,7.0,1.0,4.0,5.0,4.0,13.0,6.0,5.0,8.0,13.0,2.0,1.0,9.0,7.0,42.0


In [63]:
merged_table = pd.concat([u_test, u_control_2, u_control_1])
merged_table

country,Argentina,Australia,Belgium,Canada,Chile,France,Germany,Israel,Italy,Spain,Switzerland,Turkey,United Arab Emirates,United Kingdom (Great Britain),United States of America
group,,,,,,,,,,,,,,,
test,7.0,1.0,4.0,5.0,4.0,13.0,6.0,5.0,8.0,13.0,2.0,1.0,9.0,7.0,42.0
control_2,10.0,2.0,10.0,4.0,11.0,19.0,5.0,6.0,16.0,12.0,3.0,1.0,5.0,10.0,64.0
control_1,12.0,3.0,4.0,9.0,10.0,22.0,5.0,14.0,17.0,9.0,3.0,3.0,7.0,10.0,60.0


In [64]:
chi2, p, dof, expected = chi2_contingency(merged_table)

print(f"Статистика хи-квадрат: {chi2}")
print(f"P-значение: {p}")
print(f"Степени свободы: {dof}")
print(f"Ожидаемые частоты:\n{expected}")

Статистика хи-квадрат: 20.550060327593798
P-значение: 0.8435965188664911
Степени свободы: 28
Ожидаемые частоты:
[[ 7.47058824  1.54563895  4.63691684  4.63691684  6.44016227 13.91075051
   4.12170385  6.44016227 10.56186613  8.75862069  2.06085193  1.28803245
   5.40973631  6.95537525 42.76267748]
 [10.47058824  2.1663286   6.4989858   6.4989858   9.02636917 19.4969574
   5.77687627  9.02636917 14.80324544 12.27586207  2.88843813  1.80527383
   7.5821501   9.7484787  59.93509128]
 [11.05882353  2.28803245  6.86409736  6.86409736  9.53346856 20.59229209
   6.10141988  9.53346856 15.63488844 12.96551724  3.05070994  1.90669371
   8.00811359 10.29614604 63.30223124]]


#### Вывод: Не отклоняем нулевую гипотезу: нет статистически значимых различий в распределении по странам между группами

Объединяем данные по пользователям и их транзакциям по каждой из 3х групп и заводим столбец для обозначения группы к которой принадлежат данные

In [65]:
merged_df_test = pd.merge(users_test[['uid']], transactions_test[['uid', 'revenue' , 'product_type']], on='uid', how='left')
merged_df_test = merged_df_test.fillna(0)
merged_df_test['group'] = 'test'
merged_df_test

,uid,revenue,product_type,group
0,891152790,12597,premium_no_trial,test
1,891589239,12597,premium_no_trial,test
2,891343671,16549,trial_premium,test
3,892430988,12727,trial_premium,test
4,891919368,16536,trial_premium,test
...,...,...,...,...
132,891226275,12727,trial_premium,test
133,891718167,8164,premium_no_trial,test
134,891969984,5330,premium_no_trial,test
135,892107222,9087,trial_premium,test


In [67]:
merged_df_control_1 = pd.merge(users_control_1[['uid']], transactions_control_1_filt[['uid', 'revenue', 'product_type']], on='uid', how='left')
merged_df_control_1 = merged_df_control_1.fillna(0)
merged_df_control_1['group'] = 'C1'
merged_df_control_1

,uid,revenue,product_type,group
0,891497961,0.0,0,C1
1,892380033,4732.0,premium_no_trial,C1
2,891292845,1898.0,premium_no_trial,C1
3,892336797,6292.0,premium_no_trial,C1
4,891264903,3887.0,premium_no_trial,C1
...,...,...,...,...
188,891174534,10517.0,trial_premium,C1
189,891131061,6292.0,premium_no_trial,C1
190,891543996,6292.0,premium_no_trial,C1
191,892145004,9919.0,trial_premium,C1


In [68]:
merged_df_control_2 = pd.merge(users_control_2[['uid']], transactions_control_2_filt[['uid', 'revenue', 'product_type']], on='uid', how='left')
merged_df_control_2 = merged_df_control_2.fillna(0)
merged_df_control_2['group'] = 'C2'
merged_df_control_2

,uid,revenue,product_type,group
0,892197879,0.0,0,C2
1,892164795,16562.0,trial_premium,C2
2,892343331,2873.0,premium_no_trial,C2
3,891668253,9997.0,trial_premium,C2
4,892318245,6292.0,premium_no_trial,C2
...,...,...,...,...
177,892385232,6292.0,premium_no_trial,C2
178,892186737,4732.0,premium_no_trial,C2
179,891333651,0.0,0,C2
180,891452592,12701.0,trial_premium,C2


### Расчитываем ключевые метрики по группам

#### Рассчитываем ARPPU общее для всех купивших пользователей в заданых странах и ПС

In [69]:
rev_test_PU = merged_df_test[(merged_df_test['revenue'] > 0)]['revenue']
rev_control_1_PU = merged_df_control_1[(merged_df_control_1['revenue'] > 0)]['revenue']
rev_control_2_PU = merged_df_control_2[(merged_df_control_2['revenue'] > 0)]['revenue']
ARPPU_test = rev_test_PU.mean()
ARPPU_control_1 = rev_control_1_PU.mean()
ARPPU_control_2 = rev_control_2_PU.mean()

In [70]:
print("ARPPU в группе test:", ARPPU_test)
print("ARPPU в группе control_1:", ARPPU_control_1)
print("ARPPU в группе control_2:", ARPPU_control_2)

ARPPU в группе test: 11928.970802919708
ARPPU в группе control_1: 9203.23076923077
ARPPU в группе control_2: 8797.62


#### Рассчитываем ARPPU (средний чек на каждого платящего пользователя) для тех ктро купил премиум без пробного периода


In [71]:
rev_test_PRNT = merged_df_test[(merged_df_test['revenue'] > 0) & (merged_df_test['product_type'] == 'premium_no_trial' )]['revenue']
rev_control_1_PRNT = merged_df_control_1[(merged_df_control_1['revenue'] > 0) & (merged_df_control_1['product_type'] == 'premium_no_trial' )]['revenue']
rev_control_2_PRNT = merged_df_control_2[(merged_df_control_2['revenue'] > 0) & (merged_df_control_2['product_type'] == 'premium_no_trial' )]['revenue']
ARPPU_test_PRNT = rev_test_PRNT.mean()
ARPPU_control_1_PRNT = rev_control_1_PRNT.mean()
ARPPU_control_2_PRNT = rev_control_2_PRNT.mean()

In [72]:
print("ARPPU_PRNT в группе test:", ARPPU_test_PRNT)
print("ARPPU_PRNT в группе control_1:", ARPPU_control_1_PRNT)
print("ARPPU_PRNT в группе control_2:", ARPPU_control_2_PRNT)

ARPPU_PRNT в группе test: 10807.09589041096
ARPPU_PRNT в группе control_1: 7432.009900990099
ARPPU_PRNT в группе control_2: 7124.656565656565


#### Расчитываем конверсию

In [73]:
Conversion_test = round(int(rev_test_PRNT.count()) / int(merged_df_test.revenue.count()), 3)
Conversion_control_1 = round(int(rev_control_1_PRNT.count()) / int(merged_df_control_1.revenue.count()), 3)
Conversion_control_2 = round(int(rev_control_2_PRNT.count()) / int(merged_df_control_2.revenue.count()), 3)
print("CR в группе test:", Conversion_test)
print("CR в группе control_1:", Conversion_control_1)
print("CR в группе control_2:", Conversion_control_2)

CR в группе test: 0.533
CR в группе control_1: 0.523
CR в группе control_2: 0.544


##### ARPU- метрика обозначающия средний чек на каждого юзера

In [74]:
ARPU_test = merged_df_test.revenue.mean()
ARPU_control_1 = merged_df_control_1.revenue.mean()
ARPU_control_2= merged_df_control_2.revenue.mean()
print("ARPU в группе test:", ARPU_test)
print("ARPU в группе control_1:", ARPU_control_1)
print("ARPU в группе control_2:", ARPU_control_2)

ARPU в группе test: 11928.970802919708
ARPU в группе control_1: 8058.78756476684
ARPU в группе control_2: 7250.785714285715


#### Проверка нормальности распределения ARPPU_PRNT (тест Шапиро-Уилка) . Нулевая гипотеза - данные распределены нормально.

In [75]:
_, p_value_shapiro_с1 = stats.shapiro(rev_control_1_PRNT)
_, p_value_shapiro_с2 = stats.shapiro(rev_control_2_PRNT)
_, p_value_shapiro_test = stats.shapiro(rev_test_PRNT)

print("P-value для теста Шапиро-Уилка (группа control_1):", p_value_shapiro_с1)
print("P-value для теста Шапиро-Уилка (группа control_2):", p_value_shapiro_с2)
print("P-value для теста Шапиро-Уилка (группа тестовая):", p_value_shapiro_test)

P-value для теста Шапиро-Уилка (группа control_1): 9.791385537410418e-20
P-value для теста Шапиро-Уилка (группа control_2): 1.9083406225456128e-20
P-value для теста Шапиро-Уилка (группа тестовая): 6.457832537507054e-17


#### ВЫВОД: так как p_value меньше 0.05 , то данные распределены не нормально, следовательно используем критерий Уэлча или ANOVA

Проверка гемогенности дисперсий между группами (тест Левене)

In [76]:
statistic, p_value = levene(rev_test_PRNT, rev_control_1_PRNT, rev_control_2_PRNT)

# Выводим результаты
print(f'Статистика теста Левене: {statistic}')
print(f'p-значение: {p_value}')

Статистика теста Левене: 1.4609759087233654
p-значение: 0.23383790624446937


#### ВЫВОД: так как p_value > 0.05, следовательно дисперсии равны (гомогенны), значит проводим тест ANOVA

In [77]:
data = {
    'Test Group': rev_test_PRNT,
    'Control Group 1': rev_control_1_PRNT,
    'Control Group 2': rev_control_2_PRNT
}

In [78]:
df = pd.DataFrame(data)

In [79]:
df = df.fillna(0)

In [80]:
f_statistic, p_value = f_oneway(df['Test Group'], df['Control Group 1'], df['Control Group 2'])

# Вывод результатов
print(f'F-статистика: {f_statistic}, p-значение: {p_value}')

F-статистика: 0.1001513875485225, p-значение: 0.904718664305828


#### ВЫВОД: так как p-значение больше 0.05, мы  не отвергаем нулевую гипотезу, что указывает на то, что нет статзначимых различий в части ARPPU_PRNT между группами

#### Здесь и далее: если тест ANOVA показал, что статзначимых различий нет, то проводить попарное сравнение не целесообразно, но в этом проекте это было произведено в иследовательских целях

Проведем тест Тьюки на определения стат знач разницы в ARPPU для попарного сравнения групп. Для этого создаем датафреймы с выручкой по платящим пользователдям и обозначением группы для попарного сравнения. 

In [81]:
mc1 = merged_df_control_1[(merged_df_control_1['revenue'] > 0) & (merged_df_control_1['product_type'] == 'premium_no_trial' )]

In [82]:
mc1

,uid,revenue,product_type,group
1,892380033,4732.0,premium_no_trial,C1
2,891292845,1898.0,premium_no_trial,C1
3,892336797,6292.0,premium_no_trial,C1
4,891264903,3887.0,premium_no_trial,C1
5,891264903,3874.0,premium_no_trial,C1
...,...,...,...,...
185,891616074,6292.0,premium_no_trial,C1
186,892344951,3614.0,premium_no_trial,C1
189,891131061,6292.0,premium_no_trial,C1
190,891543996,6292.0,premium_no_trial,C1


In [83]:
mc2 = merged_df_control_2[(merged_df_control_2['revenue'] > 0) & (merged_df_control_2['product_type'] == 'premium_no_trial' )]

mc2

,uid,revenue,product_type,group
2,892343331,2873.0,premium_no_trial,C2
4,892318245,6292.0,premium_no_trial,C2
6,892334679,6461.0,premium_no_trial,C2
8,891949272,6292.0,premium_no_trial,C2
9,891291573,6292.0,premium_no_trial,C2
...,...,...,...,...
174,891883854,6461.0,premium_no_trial,C2
176,891826944,6292.0,premium_no_trial,C2
177,892385232,6292.0,premium_no_trial,C2
178,892186737,4732.0,premium_no_trial,C2


In [84]:
mtest= merged_df_test[(merged_df_test['revenue'] > 0) & (merged_df_test['product_type'] == 'premium_no_trial' )]
mtest 

,uid,revenue,product_type,group
0,891152790,12597,premium_no_trial,test
1,891589239,12597,premium_no_trial,test
5,891806436,12597,premium_no_trial,test
6,891882681,12597,premium_no_trial,test
9,892211106,4537,premium_no_trial,test
...,...,...,...,...
129,891312873,7852,premium_no_trial,test
131,891071157,9100,premium_no_trial,test
133,891718167,8164,premium_no_trial,test
134,891969984,5330,premium_no_trial,test


In [85]:
df = pd.concat([mtest, mc2, mc1], ignore_index=True)
df 

,uid,revenue,product_type,group
0,891152790,12597.0,premium_no_trial,test
1,891589239,12597.0,premium_no_trial,test
2,891806436,12597.0,premium_no_trial,test
3,891882681,12597.0,premium_no_trial,test
4,892211106,4537.0,premium_no_trial,test
...,...,...,...,...
268,891616074,6292.0,premium_no_trial,C1
269,892344951,3614.0,premium_no_trial,C1
270,891131061,6292.0,premium_no_trial,C1
271,891543996,6292.0,premium_no_trial,C1


In [86]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd


tukey = pairwise_tukeyhsd(endog=df['revenue'], groups=df['group'], alpha=0.05)
print(tukey)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
    C1     C2 -307.3533 0.9838 -4516.9221 3902.2154  False
    C1   test  3375.086 0.1924 -1197.4107 7947.5827  False
    C2   test 3682.4393 0.1435  -909.3937 8274.2723  False
----------------------------------------------------------


#### Вывод: нет статистически значимых различий между  тестовой и контрольными группами в части ARPPU_PRNT

#### Проверка нормальности распределения ARPU (тест Шапиро-Уилка) . Нулевая гипотеза - данные распределены нормально.

In [87]:
_, p_value_shapiro_с1 = stats.shapiro(merged_df_test.revenue)
_, p_value_shapiro_с2 = stats.shapiro(merged_df_control_1.revenue)
_, p_value_shapiro_test = stats.shapiro(merged_df_control_2.revenue)

print("P-value для теста Шапиро-Уилка (группа control_1):", p_value_shapiro_с1)
print("P-value для теста Шапиро-Уилка (группа control_2):", p_value_shapiro_с2)
print("P-value для теста Шапиро-Уилка (группа control_2):", p_value_shapiro_test)

P-value для теста Шапиро-Уилка (группа control_1): 3.3819336279770295e-22
P-value для теста Шапиро-Уилка (группа control_2): 1.734038130567874e-21
P-value для теста Шапиро-Уилка (группа control_2): 9.21704956077677e-22


##### ВЫВОД: так как p_value меньше 0.05 , то данные распределены не нормально


Проверка гемогенности дисперсий между группами (тест Левене)

In [88]:
statistic, p_value = levene(merged_df_test.revenue, merged_df_control_1.revenue, merged_df_control_2.revenue)

# Выводим результаты
print(f'Статистика теста Левене: {statistic}')
print(f'p-значение: {p_value}')

Статистика теста Левене: 0.3429225302180174
p-значение: 0.7098570236144619


#### ВЫВОД: так как p_value > 0.05, следовательно дисперсии равны (гомогенны) и можно провести тест ANOVA

In [89]:
data = {
    'Test Group': merged_df_test.revenue,
    'Control Group 1': merged_df_control_1.revenue,
    'Control Group 2': merged_df_control_2.revenue
}

In [90]:
df = pd.DataFrame(data)

In [91]:
df = df.fillna(0)

In [92]:
f_statistic, p_value = f_oneway(df['Test Group'], df['Control Group 1'], df['Control Group 2'])

# Вывод результатов
print(f'F-статистика: {f_statistic}, p-значение: {p_value}')

F-статистика: 1.4141133969065918, p-значение: 0.2439839122562835


#### ВЫВОД: так как p-значение больше 0.05, мы  не отвергаем нулевую гипотезу, что указывает на то, что нет статзначимых различий между группами  в части ARPU

Проведем тест Тьюки на определения стат знач разницы в ARPU для попарного сравнения групп. Для этого создаем датафреймы с выручкой по всем пользователям и обозначением группы для попарного сравнения.

In [93]:
df_ARPU  = pd.concat([merged_df_test, merged_df_control_1, merged_df_control_2], ignore_index=True)
df_ARPU 

,uid,revenue,product_type,group
0,891152790,12597.0,premium_no_trial,test
1,891589239,12597.0,premium_no_trial,test
2,891343671,16549.0,trial_premium,test
3,892430988,12727.0,trial_premium,test
4,891919368,16536.0,trial_premium,test
...,...,...,...,...
507,892385232,6292.0,premium_no_trial,C2
508,892186737,4732.0,premium_no_trial,C2
509,891333651,0.0,0,C2
510,891452592,12701.0,trial_premium,C2


In [94]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd


tukey = pairwise_tukeyhsd(endog=df_ARPU['revenue'], groups=df_ARPU['group'], alpha=0.05)
print(tukey)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff p-adj    lower      upper   reject
----------------------------------------------------------
    C1     C2 -808.0019 0.7116 -3223.5614 1607.5577  False
    C1   test 3870.1832 0.0016  1258.4158 6481.9506   True
    C2   test 4678.1851 0.0001   2033.854 7322.5162   True
----------------------------------------------------------


#### Вывод: критерий Тьюки показал статзначимые различия  в части ARPU, что противоречит результатам ANOVA.  Вероятно результаты различаются из-за относительно не большого объема выборки. Дополнительно посмотрим на практическую значимость результатов эксперимента. ARPU в тестовой группе выше на 48%, чем в контрольной группе 1 и на 69% выше, чем в контрольной группе 2, что может являться существенным результатом с точки зрения бизнеса.

#### Выявляем статзначимые различия  между группами в части конверсий.

|Cравниваем пропорции между 3 мя группами, для этого создаем df с таблицой сопряженности

In [95]:
data = {
    'конверсии': [len(rev_test_PRNT), len(rev_control_1_PRNT), len(rev_control_2_PRNT)],
    'Всего участников': [len(merged_df_test.uid.unique()), len(merged_df_control_1.uid.unique()), len(merged_df_control_2.uid.unique())]
}
data

{'конверсии': [73, 101, 99], 'Всего участников': [127, 188, 178]}

In [96]:
df = pd.DataFrame(data, index=['test', 'C1', 'C2'])
df

,конверсии,Всего участников
test,73,127
C1,101,188
C2,99,178


In [97]:
chi2, p, dof, expected = chi2_contingency(df)

print(f'Chi-squared statistic: {chi2}')
print(f'p-value: {p}')
print(f'Degrees of freedom: {dof}')
print(f'Expected frequencies:\n{expected}')

Chi-squared statistic: 0.12601326036386226
p-value: 0.9389372483437173
Degrees of freedom: 2
Expected frequencies:
[[ 71.27937337 128.72062663]
 [102.99869452 186.00130548]
 [ 98.72193211 178.27806789]]


#### ВЫВОД: p_value больше 0.05, следовательно  мы не можем отвергнуть нулевую гипотезу, и можно считать, что конверсии статзначимо не различаются по распределению

### ОСНОВНОЙ ВЫВОД: Так как большинство статтестов не выявили статзначимых различий между тестовой и контрольными группами, говорить об эффективности акции сложно, однако тест Тьюки показал статразличия между группами в части ARPU и так как ARPU в тестовой группе отличается значительно (практическая значимость отличается значительно в процентном соотношении), можно принять это во внимание и признать тест удачным.